# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**описание датасета**  
- сalls — количество звонков,  
- minutes — суммарная длительность звонков в минутах,  
- messages — количество sms-сообщений,  
- mb_used — израсходованный интернет-трафик в Мб,  
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('users_behavior.csv')

df.head(2)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.duplicated().sum()

0

пропусков нет, дубликатов нет. Некоторые типы излишне float, переведем в int.

In [5]:
# переведем некоторые типы к int
df[['calls', 'messages', 'is_ultra']] = df[['calls', 'messages', 'is_ultra']].astype(int)

In [6]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [7]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Выборка не сбалансирована, с тарифом ultra - примерно треть клиентов

## Разбейте данные на выборки

Так как мы применяем кроссвалидацию в сетке GridSearchCV, то разобъем данные на две выборки: тренировочную и тестовую.

In [8]:

X_train, X_test, y_train, y_test = train_test_split(df.drop('is_ultra', axis=1), df['is_ultra'],
                                              random_state=12345,
                                              test_size=0.3)

In [9]:
# проверка
print("Получили данные в следующей пропорции:")
print(round(100*X_train.shape[0]/df.shape[0],0), ":",
      # round(100*X_val.shape[0]/df.shape[0],0), ":",
      round(100*X_test.shape[0]/df.shape[0],0))

Получили данные в следующей пропорции:
70.0 : 30.0


## Исследуйте модели

для подбора гиперпараметров велосипед с циклами изобретать не будем, воспользуемся старым добрым GridSearchCV
Выведем на экран лучшие гиперпараметры, и лучшую точность при подборе параметров.

In [10]:
# 1. Классификатор по градиентному бустингу
gbc = GradientBoostingClassifier(random_state=666)

param_grid = {'learning_rate':[0.001, 0.01],
              'n_estimators':[x for x in range(90,111,20)],
              'max_depth':[x for x in range(2,4)],
              'min_samples_split':[x for x in range(2,4)],
              'min_samples_leaf':[1,2],
              'subsample':[0.6, 0.8],
              }

grid_search_gbc = GridSearchCV(gbc, param_grid, scoring = 'accuracy', n_jobs=-1, cv=5)
grid_search_gbc.fit(X_train, y_train)

print(grid_search_gbc.best_params_)
print(grid_search_gbc.best_score_)

{'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 110, 'subsample': 0.6}
0.7972551348676069


In [11]:
# 2. Случайный лес
rfc = RandomForestClassifier(random_state=666)

param_grid = {'n_estimators':[100, 200],
              'criterion':['gini', 'entropy'],
              'max_depth':[x for x in range(12,16,3)],
              'min_samples_split':[x for x in range(2,4)],
              'min_samples_leaf':[x for x in range(3,5)],
              }

grid_search_rfc = GridSearchCV(rfc, param_grid, scoring = 'accuracy', n_jobs=-1, cv=5)
grid_search_rfc.fit(X_train, y_train)

print(grid_search_rfc.best_params_)
print(grid_search_rfc.best_score_)


{'criterion': 'entropy', 'max_depth': 12, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 100}
0.8177035387280377


In [12]:
# 3. Логистическая регрессия
lr = LogisticRegression(random_state=666)

param_grid = {'penalty':['l1', 'l2'],
              'C': [0.1, 0.25],
              'fit_intercept':[True, False],
              #'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
              'multi_class':['auto', 'ovr', 'multinomial'],
              #'max_iter':[100, 500, 2000]
              }

grid_search_lr = GridSearchCV(lr, param_grid, scoring = 'accuracy', n_jobs=-1, cv=5)
grid_search_lr.fit(X_train, y_train)

print(grid_search_lr.best_params_)
print(grid_search_lr.best_score_)

{'C': 0.1, 'fit_intercept': True, 'multi_class': 'auto', 'penalty': 'l2'}
0.7341163078445929


In [13]:
# 4. Дерево решений
dtc = DecisionTreeClassifier(criterion='entropy', max_depth=4)

param_grid = {'criterion':['gini', 'entropy'],
              'splitter':['best', 'random'],
              'max_depth':[x for x in range(6,8)],
              'min_samples_split':[x for x in range(3,5)],
              'min_samples_leaf':[x for x in range(8,10)],
              }

grid_search_dtc = GridSearchCV(dtc, param_grid, scoring = 'accuracy', n_jobs=-1, cv=5)
grid_search_dtc.fit(X_train, y_train)

print(grid_search_dtc.best_params_)
print(grid_search_dtc.best_score_)

{'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 8, 'min_samples_split': 3, 'splitter': 'best'}
0.800807720861173


**предварительные выводы**  
Итак, использовали сетку для подбора гиперпараметров, определили лучшую модель и гиперпараметры для нее.
первое место занимает случайный лес, последнее - логистическая регрессия. Дальше будем работать со случайным лесом.

## Проверьте модель на тестовой выборке  

Посмототрим на результат выбранной модели на тестовой выборке для выбранной модели

In [14]:
print('Случайный лес:')
print('На тренировочной выборке:', accuracy_score(grid_search_rfc.predict(X_train), y_train).round(3))
print('На тестовой выборке:', accuracy_score(grid_search_rfc.predict(X_test), y_test).round(3))

Случайный лес:
На тренировочной выборке: 0.888
На тестовой выборке: 0.803


Итак, у нас получилось добиться неплохого качества (accuracy = 0.8 на тестовой выборке) в выбранной модели случайного леса.

## (бонус) Проверьте модели на адекватность

In [15]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=666)

dummy_clf.fit(X_train, y_train)

print('точность думми на валидационной выборке', dummy_clf.score(X_train, y_train))
print('точность думми на тестовой выборке', dummy_clf.score(X_test, y_test))

точность думми на валидационной выборке 0.6927523343708315
точность думми на тестовой выборке 0.6953367875647668


Как видим, точность нашей модели выше чем у думми, поэтому можем считать нашу модель адекватной и эффективной

## Выводы

- Данные чистые, без пропусков, дубликатов и аномалий. Единственное, что мы сделали - чуть облегчили датасет, за счет изменения типа некоторых данных на int  
- Выбрали четыре модели: GradientBoostingClassifier, RandomForestClassifier, LogisticRegression, DecisionTreeClassifier. Подобрали для них оптимальные гиперпараметры. Лучшими себя как обычно показали Граддиентный бустинг и случайный лес. Но случайный лес чуть лучше, поэтому выбрали его. Над регрессией надо шаманить признаки, чтобы получить более достойный вариант модели. 
- Accuracy на тестовой выборке для модели RandomForestClassifier получили на уровне 0.8
- Сравнили с baseline моделью DummyClassifier для оценки моделей на адекватность. Адекватность и эффективность подтвердили.
- Дальнейшие рекомендации: 
    - можно попробовать добавить StandartScaler, посмотреть на поведение моделей (на случайный лес он обычно не влияет, а вот на логистическую регрессию может повлиять в лучшую сторону);
    - можно попробовать сделать стеккинг;
    - можно попробовать DL, но это уже совсем другая история...